In [1]:
import polars as pl
import lancedb

/home/evan/Documents/hypersync_lancedb/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cd ../

/home/evan/Documents/hypersync_lancedb


/home/evan/Documents/hypersync_lancedb/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
table_name = "blocks"
db: lancedb.DBConnection = lancedb.connect("data")

In [4]:
# Try to open and merge data into existing table.
blocks_table: lancedb.table = db.open_table(table_name)

blocks_index: pl.DataFrame = (
    blocks_table.to_polars()
    .select("block_number")
    .sort(by="block_number", descending=True)
    .collect()["block_number"]
)

In [5]:
blocks_index.shape

(55002,)

In [6]:
txs_table: lancedb.table = db.open_table("transactions")
txs_index: pl.DataFrame = (
    txs_table.to_polars()
    .sort(by="block_number", descending=True)
    .drop("input", "logs_bloom", "sighash")
    .collect()
)

In [7]:
txs_index.select("block_number").unique().shape

(54973, 1)

### Hypersync parquet read

Verifies that the data is the same

In [6]:
blocks_lf = pl.scan_parquet("data/blocks.parquet")
txs_lf = pl.scan_parquet("data/transactions.parquet")

In [9]:
blocks_df = blocks_lf.sort(by="number", descending=True).collect()

In [11]:
blocks_df.shape

(55002, 19)

In [12]:
txs_df = (
    txs_lf.sort(by="block_number", descending=True)
    .drop("input", "logs_bloom", "sighash")
    .collect()
)

In [14]:
txs_df.select("block_number").unique().shape

(54973, 1)